In [1]:
import nltk 
import numpy as np
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
# import warnings
# warnings.filterwarnings('ignore')
import time

In [2]:
# Explore dataset
pd.read_csv("Combined_News_DJIA.csv")

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

<h4>Read in the dataset used to train and test the sentiment analyzer</h4>

In [3]:
original_df = pd.read_csv("Reddit_Data.csv") # This dataset contains ~37k arbritrary reddit comment along with their sentiment
print(original_df.shape)
original_df = original_df.dropna(axis=0) # Remove NA's
print(original_df.shape)
# df = df.head(25000)
original_df

(37249, 2)
(37149, 2)


,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1
...,...,...
37244,jesus,0
37245,kya bhai pure saal chutiya banaya modi aur jab...,1
37246,downvote karna tha par upvote hogaya,0
37247,haha nice,1


<h4>Gather in the stop words and put into a set</h4>

In [4]:
# Gather stop words
stopwords = set(w.rstrip() for w in open('stopwords.txt'))
stopwords

{'a',
 'about',
 'above',
 'across',
 'after',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'among',
 'an',
 'and',
 'another',
 'any',
 'anybody',
 'anyone',
 'anything',
 'anywhere',
 'are',
 'area',
 'areas',
 'around',
 'as',
 'ask',
 'asked',
 'asking',
 'asks',
 'at',
 'away',
 'b',
 'back',
 'backed',
 'backing',
 'backs',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'been',
 'before',
 'began',
 'behind',
 'being',
 'beings',
 'best',
 'better',
 'between',
 'big',
 'both',
 'but',
 'by',
 'c',
 'came',
 'can',
 'cannot',
 'case',
 'cases',
 'certain',
 'certainly',
 'clear',
 'clearly',
 'come',
 'could',
 'd',
 'did',
 'differ',
 'different',
 'differently',
 'do',
 'does',
 'done',
 'down',
 'downed',
 'downing',
 'downs',
 'during',
 'e',
 'each',
 'early',
 'either',
 'end',
 'ended',
 'ending',
 'ends',
 'enough',
 'even',
 'evenly',
 'ever',
 'every',
 'everybody',
 'everyone',
 'everything',
 'everyw

<h4>This function accepts a dataframe as a parameter and tokenizes every entry in the dataframe and adds it to a tokens list. It also removes tokens that are stopwords or less than 3 characters. It then adds all the unique words to a word index map where the key is the word/token and the value is the index of where it first occurred. The purpose of this word index map is to allow me to map a word to a position and compute the frequency of its occurrence. Lastly, the function returns the tokens list and the word index map</h4>

In [5]:
# Loops through each comment and tokenize it and remove stop words. Also, create word index map to compute word frequencies
def tokenize_comments(df):
    wordnet_lemmatizer = WordNetLemmatizer() # Create a lemmatizer object to transform word into their base form (i.e dogs and dog becomes the same word
    tokens_list = [] 
    word_index_map = {}
    i=0
    for index, row in df.iterrows():
        tokens = nltk.tokenize.word_tokenize(row["clean_comment"])# Tokenize the comments
        tokens = [t for t in tokens if len(t) > 2]
        tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Convert words into their base form
        tokens = [t for t in tokens if t not in stopwords] # Only adds words to the tokens if they are not stopwords and the length of the string is > 2
        for token in tokens: # Loop through and get add each token/word to word index map
            if token not in word_index_map:
                word_index_map[token] = i
                i += 1
        tokens_list.append(tokens)
    return tokens_list, word_index_map # return tokenize list and the word index map

In [6]:
# Tokenize comments
tokens_list, word_index_map = tokenize_comments(original_df)

In [7]:
print("Tokens list without label \n")
tokens_list

Tokens list without label 



[['family',
  'mormon',
  'tried',
  'explain',
  'stare',
  'puzzled',
  'time',
  'time',
  'strange',
  'creature',
  'nonetheless',
  'admire',
  'patience',
  'calmness',
  'equanimity',
  'acceptance',
  'compassion',
  'developed',
  'buddhism',
  'teach'],
 ['buddhism',
  'ha',
  'lot',
  'compatible',
  'christianity',
  'especially',
  'considering',
  'sin',
  'suffering',
  'suffering',
  'caused',
  'shouldn',
  'getting',
  'wrong',
  'christian',
  'mean',
  'don',
  'coincide',
  'god',
  'coincide',
  'aid',
  'jesus',
  'buddhism',
  'seen',
  'proof',
  'god',
  'mighty',
  'omnipotence',
  'christian',
  'lucky',
  'christ',
  'else',
  'christian',
  'believe',
  'god',
  'grace',
  'salvation',
  'buddhism',
  'god',
  'grace',
  'help',
  'study',
  'jesus',
  'buddha',
  'ha',
  'similar',
  'claim',
  'rich',
  'getting',
  'heaven',
  'joke',
  'basically',
  'advocating',
  'rid',
  'ourselves',
  'material',
  'possession',
  'distinctly',
  'remembered',
  

In [8]:
print("Word index map")
word_index_map

Word index map


{'family': 0,
 'mormon': 1,
 'tried': 2,
 'explain': 3,
 'stare': 4,
 'puzzled': 5,
 'time': 6,
 'strange': 7,
 'creature': 8,
 'nonetheless': 9,
 'admire': 10,
 'patience': 11,
 'calmness': 12,
 'equanimity': 13,
 'acceptance': 14,
 'compassion': 15,
 'developed': 16,
 'buddhism': 17,
 'teach': 18,
 'ha': 19,
 'lot': 20,
 'compatible': 21,
 'christianity': 22,
 'especially': 23,
 'considering': 24,
 'sin': 25,
 'suffering': 26,
 'caused': 27,
 'shouldn': 28,
 'getting': 29,
 'wrong': 30,
 'christian': 31,
 'mean': 32,
 'don': 33,
 'coincide': 34,
 'god': 35,
 'aid': 36,
 'jesus': 37,
 'seen': 38,
 'proof': 39,
 'mighty': 40,
 'omnipotence': 41,
 'lucky': 42,
 'christ': 43,
 'else': 44,
 'believe': 45,
 'grace': 46,
 'salvation': 47,
 'help': 48,
 'study': 49,
 'buddha': 50,
 'similar': 51,
 'claim': 52,
 'rich': 53,
 'heaven': 54,
 'joke': 55,
 'basically': 56,
 'advocating': 57,
 'rid': 58,
 'ourselves': 59,
 'material': 60,
 'possession': 61,
 'distinctly': 62,
 'remembered': 63,
 '

<h4>The function "normalize_tokens" accepts the tokens list and the word index map returned for the "tokenize_comments" function and normalizes each token according to its token list.</h4>

In [9]:
# Structure and normalize each tokens list
# np.seterr(divide = 'ignore') # Ignore divide by zero warning
def normalize_tokens(tokens_list, word_index_map):
    normalized_tokens_list = []
    for token_list in tokens_list: # Loop each tokens list (comment that has been tokenized)
        x = np.zeros(len(word_index_map) + 1)
        for token in token_list: # Loop each token in the comment 
            i = word_index_map[token] # Get (first) occurence of word. Arbritray number but needed for organization
            x[i] += 1 # Increment
        x = np.divide(x,x.sum()) # Divide the frequency vector by total sum allowing for us to investigate it words total usage in the comment
        normalized_tokens_list.append(x)
    return normalized_tokens_list

In [10]:
# Normalize token
normalized_tokens_list = normalize_tokens(tokens_list, word_index_map)

/home/luis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


In [11]:
print('Normalized tokens. (i.e. If there are twenty words in the first tokens list and the first word "family" appeared once so it has a value of 1/20=0.05)')
normalized_tokens_list

Normalized tokens. (i.e. If there are twenty words in the first tokens list and the first word "family" appeared once so it has a value of 1/20=0.05)


[array([0.05, 0.05, 0.05, ..., 0.  , 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.02941176, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.        , 0.        , 0.01265823, ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.02564103, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 

<h4>Function that appends the target variable which is the sentiment label back onto its respective tokens list </h4>

In [12]:
def attach_labels(tokens_list, labels):
    for i in range(len(tokens_list)):
        tokens_list[i] = np.append(tokens_list[i], np.array(labels)[i])
    return tokens_list

<h4>Attach the sentiment labels back to its respective comment</h4>

In [13]:
new_tokens_list = attach_labels(normalized_tokens_list, original_df["category"])

<h4>Convert the list of np arrays into np mats and then into dataframe</h4>

In [14]:
new_df = pd.DataFrame(np.mat(new_tokens_list))

<h4>We must drop the rows that contain null/na values that came from removing stopwords and words less than 3 chars</h4>

In [15]:
# Drop row that contains NA values
new_df = new_df.dropna(axis=0)

<h4>Display new dataframe</h4>

In [16]:
new_df # Comment x Term matrix

,0,1,2,3,4,5,6,7,8,9,...,48924,48925,48926,48927,48928,48929,48930,48931,48932,48933
0,0.050000,0.05,0.05,0.050000,0.05,0.05,0.1,0.05,0.05,0.05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.029412,0.00,0.00,0.029412,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0
3,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37144,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37145,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
37146,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37147,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


<h4>Extract the target variable from the new dataframe and remove it</h4>

In [17]:
df_target = new_df[new_df.columns[-1]] # Extract target variable 
df = new_df.drop(new_df.columns[-1], axis = 1) # Drop target column from data

<h4>Split the dataset</h4>

In [18]:
from sklearn.model_selection import train_test_split
df_train, df_test, df_train_target, df_test_target = train_test_split(df, df_target, test_size=0.2, random_state=33)

<h4>Create a sentiment analyzer using logistic regression</h4>

In [19]:
start_time = time.time()

sentiment_analyzer_lr = LogisticRegression(max_iter = 500)
sentiment_analyzer_lr.fit(df_train, df_train_target)
sentiment_analyzer_lrpreds_test = sentiment_analyzer_lr.predict(df_test)
# Run 5-fold validation
cv_scores = cross_val_score(sentiment_analyzer_lr, df_test, df_test_target, cv=5)
print("Accuracy for logistic regression classifier")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", sentiment_analyzer_lr.score(df_train, df_train_target))
print("Score on Test: ", sentiment_analyzer_lr.score(df_test, df_test_target))


print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for logistic regression classifier
Overall Accuracy on X-Val: 0.56 (+/- 0.02)
Score on Training:  0.7173927818935635
Score on Test:  0.667527524806307
--- 251.32787466049194 seconds ---


<h4> Once the sentiment analyzer is built, let's now build the DJIA prediction system</h4>

In [20]:
# Gather in the dataset
data = pd.read_csv("Combined_News_DJIA.csv")
data
print("Dimension before removing NA's: " + str(data.shape))

Dimension before removing NA's: (1989, 27)


<h4>Drop rows that contain NA and reindex the dataframe after dropping the rows</h4>

In [21]:
# Drop na's and reindex dataframe
data = data.dropna()
print("Dimension after removing NA's: " + str(data.shape))
data = data.reset_index(drop=True) # Reindex the dataframe after dropping column
data

Dimension after removing NA's: (1986, 27)


,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

<h4>Loop through each column and replace the unneccessary tag "b."</h4>

In [22]:
# Loop through each column and replace the unneccessary tag "b."
for column in data.columns[2:]: 
    data[column] = data[column].str.replace('b.', ' ', regex=True)

<h4> This function accepts a string of words (i.e. sentence/headline) tokenizes it, remove tokens that are less than 2 characters, remove tokens that are stopwords after lemmatizing the token </h4>

In [23]:
def tokenize_string(input_str):
    wordnet_lemmatizer = WordNetLemmatizer() 
    tokens = nltk.tokenize.word_tokenize(input_str) # Tokenize string
    tokens = [t for t in tokens if len(t) > 2] # Remove words less than 2 chars
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Convert words into their base form
    tokens = [t for t in tokens if t not in stopwords] # Only adds words to the tokens if they are not stopwords and the length of the string is > 2
    return tokens

<h4>This function creates a vector where each index represents a word in the word dictionary. At each index, the value is the frequency of that word occuring in the token's list passed in the function. Lastly, divide the frequency of each of entry in the vector by the total number of tokens (tokens in the token list) </h4>

In [24]:
def normalize_tokenized_string(tokens):
    x = np.zeros(len(word_index_map) + 1)
    for token in tokens: # Loop each token in the comment 
        if token in word_index_map: # We can't analyze words that we haven't used for training
            i = word_index_map[token] # Get (first) occurence of word. Arbritray number but needed for organization
            x[i] += 1
    if x.sum() != 0: # No words are in this headline have sentiment analyzer been trained on
        x = np.divide(x,x.sum())
    return x

<h4>Function that rebuilds the original dataset that was made up of top 25 headlines for each day from June 2008 to July 2016 with it sentiment label representation. To do this, I pass a sentiment analyzer along with the dataset to make predictions of the sentiment of each headlines in every data object. Return newly created matrix. </h4>

In [25]:
def reconstruct(data, sentiment_analyzer):
    mat = np.zeros(shape=(data.shape[0], data.shape[1]-2)) # Create matrix to hold the data
    position = 0 # Pointer for our data
    for index, row in data.iterrows():
        predictions = [] # The current row's prediction list (i.e. That day's top 25 headlines)
        for i in range(2, len(data.columns)): # Loop through each entry in the current row
            current_comment = data[data.columns[i]][index] # Get the current comment in the row
            tokens = tokenize_string(current_comment) # Tokenize the comment
            normalized_tokens = normalize_tokenized_string(tokens) # Normalize the tokens by frequency
            prediction = sentiment_analyzer.predict([normalized_tokens]) # Predict the sentiment label for this entry 
            predictions.append(prediction[0]) # Add the label to the vector
        mat[position, :] = predictions # Add the vector (the row's sentiment label representation) into the matrix
        position += 1 # Increment row position
    return mat

<h4>Reconstruct the original reddit dataset to be represented by its
entries sentiment label and then convert the matrix it into a dataframe</h4>

In [26]:
df2 = pd.DataFrame(reconstruct(data, sentiment_analyzer_lr)) 
df2 # Display new dataframe 

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,-1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,-1.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,-1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,0.0,1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1982,0.0,1.0,0.0,1.0,1.0,-1.0,0.0,1.0,0.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
1983,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,1.0,-1.0,0.0,1.0,1.0,0.0
1984,1.0,-1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<h4>Now, let's take the reconstructed dataframe and create a classification model that predicts if the DJIA close value increases or decreases from the opening value.</h4>

In [27]:
# Split the dataset into train and test set
df2_train, df2_test, df2_train_target, df2_test_target = train_test_split(df2, data["Label"], test_size=0.3, random_state=33)

<h4>Create a logistic regression model to predict the behavior of the DJIA</h4>

In [28]:
start_time = time.time()

# Let's create logistic regression classifer to the behavior of the DJIA closing value
from sklearn.linear_model import LogisticRegression
lrclf = LogisticRegression(max_iter=500)
lrclf.fit(df2_train, df2_train_target)
lrpreds_test = lrclf.predict(df2_test)
# Run 10-fold validation
cv_scores = cross_val_score(lrclf, df2_test, df2_test_target, cv=5)
print("Accuracy for logistic regression classifier")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", lrclf.score(df2_train, df2_train_target))
print("Score on Test: ", lrclf.score(df2_test, df2_test_target))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for logistic regression classifier
Overall Accuracy on X-Val: 0.48 (+/- 0.08)
Score on Training:  0.5625899280575539
Score on Test:  0.5151006711409396
--- 0.03761124610900879 seconds ---


<h4>Let's see if data performs better on the top 5 headlines than the top 25 headlines</h4>

In [29]:
# Let's trim the dataframe with just the top 5 blog post headlines 
df3 = df2
df3.drop(df3.iloc[:, 5:], inplace = True, axis=1)
df3

,0,1,2,3,4
0,1.0,1.0,0.0,1.0,1.0
1,1.0,0.0,1.0,0.0,1.0
2,-1.0,1.0,0.0,1.0,0.0
3,1.0,0.0,-1.0,1.0,1.0
4,1.0,0.0,1.0,-1.0,0.0
...,...,...,...,...,...
1981,0.0,1.0,0.0,0.0,1.0
1982,0.0,1.0,0.0,1.0,1.0
1983,0.0,0.0,0.0,0.0,1.0
1984,1.0,-1.0,0.0,1.0,0.0


<h4>Split the dataset into train and test set</h4>

In [30]:
df3_train, df3_test, df3_train_target, df3_test_target = train_test_split(df3, data["Label"], test_size=0.3, random_state=33)

<h4>Create a logisic regression model</h4>

In [31]:
start_time = time.time()

# Create another logistic regression clf with just the top 5 headlines
lrclf2 = LogisticRegression(max_iter=500)
lrclf2.fit(df3_train, df3_train_target)
lrpreds_test = lrclf.predict(df2_test)
# Run 10-fold validation
cv_scores = cross_val_score(lrclf, df3_test, df3_test_target, cv=5)
print("Accuracy for logistic regression classifier (with just the top 5 headlines)")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", lrclf2.score(df3_train, df3_train_target))
print("Score on Test: ", lrclf2.score(df3_test, df3_test_target))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for logistic regression classifier (with just the top 5 headlines)
Overall Accuracy on X-Val: 0.46 (+/- 0.06)
Score on Training:  0.5410071942446043
Score on Test:  0.5184563758389261
--- 0.0272829532623291 seconds ---


<h4>Create a decision tree model</h4>

In [32]:
start_time = time.time()

# Create decision tree classifier and train the classifier
from sklearn import tree
treeclf = tree.DecisionTreeClassifier(criterion='gini')
treeclf = treeclf.fit(df2_train, df2_train_target)
# Compute predictions
treepreds_test = treeclf.predict(df2_test)
# Run 5-fold validation
cv_scores = cross_val_score(treeclf, df2_train, df2_train_target, cv=10)
print("Accuracy for decision tree classifier")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", treeclf.score(df2_train, df2_train_target))
print("Score on Test: ", treeclf.score(df2_test, df2_test_target))
## Possibly overfitting the training set ##

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for decision tree classifier
Overall Accuracy on X-Val: 0.50 (+/- 0.11)
Score on Training:  1.0
Score on Test:  0.5
--- 0.10956621170043945 seconds ---


<h4>Create a linear discriminant model</h4>

In [33]:
start_time = time.time()

# #  Create linear discriminant analysis classifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
ldclf = LinearDiscriminantAnalysis()
# Compute predictions
ldclf = ldclf.fit(df2_train, df2_train_target)
ldpreds_test = ldclf.predict(df2_test)
# Run 5-fold validation
cv_scores = cross_val_score(ldclf, df2_test, df2_test_target, cv=5)
print("Accuracy for linear discriminant analysis classifier")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", ldclf.score(df2_train, df2_train_target))
print("Score on Test: ", ldclf.score(df2_test, df2_test_target))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for linear discriminant analysis classifier
Overall Accuracy on X-Val: 0.48 (+/- 0.08)
Score on Training:  0.5618705035971223
Score on Test:  0.5151006711409396
--- 0.04892468452453613 seconds ---


<h4>Create a SVM model</h4>

In [34]:
start_time = time.time()
from sklearn.svm import SVC
# Create a kernelized SVM by using RBF (radial basis function)
svm_k = SVC(kernel='rbf', C=1E6)
svm_k.fit(df2_train, df2_train_target)
cv_scores = cross_val_score(svm_k, df2_test, df2_test_target, cv=5)
print("Accuracy for support vector machine classifier")
print("Overall Accuracy on X-Val: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))
print("Score on Training: ", svm_k.score(df2_train, df2_train_target))
print("Score on Test: ", svm_k.score(df2_test, df2_test_target))

print("--- %s seconds ---" % (time.time() - start_time))

Accuracy for support vector machine classifier
Overall Accuracy on X-Val: 0.49 (+/- 0.06)
Score on Training:  1.0
Score on Test:  0.4664429530201342
--- 0.29219889640808105 seconds ---
